In [ ]:
from IPython.core.display import Image
from numpy.lib.function_base import append
import numpy as np
from google.colab.patches import cv2_imshow
import cv2 # 導入OpenCV函式庫
from google.colab import files
import matplotlib.pyplot as plt

# img = cv2.imread('toCalibrated.jpg')
# cv2_imshow(img)

# choose points================================================================
pts_a = np.array([[212,218],[266,460],[384,347],[356,143]])
pts_b = np.array([[458,127],[475,325],[671,347],[676,135]])
pts_c = np.array([[352,543],[593,594],[654,446],[453,416]])
local_c = np.array([[0,0],[0,30],[30,30],[30,0]])
# =============================================================================

# findHomography of each plant=================================================
ha ,status= cv2.findHomography(local_c, pts_a)
hb ,status= cv2.findHomography(local_c, pts_b)
hc ,status= cv2.findHomography(local_c, pts_c)
# =============================================================================

# calculate omega==============================================================
hap = ha.T
hbp = hb.T
hcp = hc.T

A = np.array([[hap[0,0]*hap[1,0], hap[0,0]*hap[1,1]+hap[0,1]*hap[1,0], hap[0,0]*hap[1,2]+hap[0,2]*hap[1,0], hap[0,1]*hap[1,1], hap[0,1]*hap[1,2]+hap[0,2]*hap[1,1], hap[0,2]*hap[1,2]],
             [hap[0,0]**2-hap[1,0]**2, 2*(hap[0,0]*hap[0,1]-hap[1,0]*hap[1,1]), 2*(hap[0,0]*hap[0,2]-hap[1,0]*hap[1,2]), hap[0,1]**2-hap[1,1]**2, 2*(hap[0,1]*hap[0,2]-hap[1,1]*hap[1,2]), hap[0,2]**2-hap[1,2]**2],
             [hbp[0,0]*hbp[1,0], hbp[0,0]*hbp[1,1]+hbp[0,1]*hbp[1,0], hbp[0,0]*hbp[1,2]+hbp[0,2]*hbp[1,0], hbp[0,1]*hbp[1,1], hbp[0,1]*hbp[1,2]+hbp[0,2]*hbp[1,1], hbp[0,2]*hbp[1,2]],
             [hbp[0,0]**2-hbp[1,0]**2, 2*(hbp[0,0]*hbp[0,1]-hbp[1,0]*hbp[1,1]), 2*(hbp[0,0]*hbp[0,2]-hbp[1,0]*hbp[1,2]), hbp[0,1]**2-hbp[1,1]**2, 2*(hbp[0,1]*hbp[0,2]-hbp[1,1]*hbp[1,2]), hbp[0,2]**2-hbp[1,2]**2],
             [hcp[0,0]*hcp[1,0], hcp[0,0]*hcp[1,1]+hcp[0,1]*hcp[1,0], hcp[0,0]*hcp[1,2]+hcp[0,2]*hcp[1,0], hcp[0,1]*hcp[1,1], hcp[0,1]*hcp[1,2]+hcp[0,2]*hcp[1,1], hcp[0,2]*hcp[1,2]],
             [hcp[0,0]**2-hcp[1,0]**2, 2*(hcp[0,0]*hcp[0,1]-hcp[1,0]*hcp[1,1]), 2*(hcp[0,0]*hcp[0,2]-hcp[1,0]*hcp[1,2]), hcp[0,1]**2-hcp[1,1]**2, 2*(hcp[0,1]*hcp[0,2]-hcp[1,1]*hcp[1,2]), hcp[0,2]**2-hcp[1,2]**2]])

U, S, Vt = np.linalg.svd(A)
V = Vt.T

W = np.vstack((V[0:3,5],np.vstack((np.hstack((V[1,5],V[3:5,5])),np.hstack((V[2,5],V[4:6,5]))))))
inW = np.linalg.inv(W)
NinW = inW/inW[2,2] #normalized let (3,3) = 1
# =============================================================================

# calculate K==================================================================
c = NinW[0,2]
e = NinW[1,2]
d = np.sqrt(NinW[1,1]-e**2)
a = np.sqrt(NinW[0,0]-c**2)
b = (NinW[0,1]-c*e)/d
b0 = 0

K = np.array([[a,b0,c],
              [0,d,e],
              [0,0,1]])
print('intrinsic parameter K matrix:\n',K)
# =============================================================================


# calculate RT==================================================================
zero = np.zeros(4).astype(int)
X = np.array([[40,0,40,1],[40,0,10,1],[40,10,0,1],[40,40,0,1],
              [0,10,40,1],[0,10,10,1],[0,40,10,1]])
uv = np.array([[212,218,1],[266,460,1],[352,543,1],[593,594,1],
              [458,127,1],[475,325,1],[671,347,1]])

# find P
Puv = np.zeros(shape=(14,12))
for i in range(7):
  Puv[2*i] = np.concatenate((X[i], zero, -uv[i,0]*X[i]))
  Puv[2*i+1] = np.concatenate((zero, X[i], -uv[i,1]*X[i]))

Up, Sp, Vpt = np.linalg.svd(Puv)
Vp = Vpt.T

P = np.vstack((np.vstack((Vp[0:4,11],Vp[4:8,11])),Vp[8:12,11]))

P = P/P[2,3]

inK = np.linalg.inv(K)
RT = inK.dot(P)
RT = np.array(RT)

real_RT = np.array(RT/(np.linalg.norm([RT[0,0],RT[1,0],RT[2,0]])))
print('extrinsic parameter RT matrix:\n',real_RT)